<a href="https://colab.research.google.com/github/jalorenzo/SparkNotebookColab/blob/master/BDF_03_Working_with_DataFrames.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#00 - Configuration of Apache Spark on Collaboratory


###Installing Java, Spark, and Findspark


---


This code installs Apache Spark 2.4.4, Java 8, and [Findspark](https://github.com/minrk/findspark), a library that makes it easy for Python to find Spark.

In [ ]:
!apt-get update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget  http://apache.osuosl.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz  
!tar xf spark-3.3.1-bin-hadoop3.tgz  
!rm spark-3.3.1-bin-hadoop3.tgz    
!pip install -q findspark

### Set Environment Variables
Set the locations where Spark and Java are installed.

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark/"
os.environ["DRIVE_DATA"] = "/content/gdrive/My Drive/Enseignement/2022-2023/ING3/HPDA/BigDataFrameworks/data/"

!rm /content/spark
!ln -s /content/spark-3.3.1-bin-hadoop3 /content/spark
!export SPARK_HOME=/content/spark
!export PATH=$PATH:$SPARK_HOME/bin:$SPARK_HOME/sbin
!echo $SPARK_HOME
!env |grep  "DRIVE_DATA"

### Start a SparkSession
This will start a local Spark session.

In [1]:
!python -V

#import findspark
#findspark.init()

from pyspark import SparkContext
sc = SparkContext.getOrCreate()

# Example: shows the PySpark version
print("PySpark version {0}".format(sc.version))

# Example: parallelise an array and show the 2 first elements
sc.parallelize([2, 3, 4, 5, 6]).cache().take(2)

Python 3.9.2


/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/23 13:21:32 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
PySpark version 3.3.1


[2, 3]

In [2]:
from pyspark.sql import SparkSession
# We create a SparkSession object (or we retrieve it if it is already created)
spark = SparkSession \
.builder \
.appName("My application") \
.config("spark.some.config.option", "some-value") \
.master("local[4]") \
.getOrCreate()
# We get the SparkContext
sc = spark.sparkContext

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/gdrive')



---


# 03 - Working with DataFrames

## Introduction to DataFrames
We will see:

  - How to create a DataFrame
  - Basic operations on DataFrames
      - Show rows
      - Select columns
      - Rename, add and delete columns
      - Delete null values and duplicated rows
      - Replace values
  - Save DataFrames in different formats

## Creating DataFrames
A DataFrame can be created in different ways:

  - From a data sequence
  - From Row-type objects 
  - From an RDD or a DataSet
  - Reading data from a file
      - Like in Hadoop, Spark supports different filesystems: local, HDFS, Amazon S3
          - By and large, it supports any data source that can be read with Hadoop
      - Spark can access different types of files: plain text, CSV, JSON, [Parquet](https://parquet.apache.org/), [ORC](https://orc.apache.org/), Sequence, etc
        -   It also supports compressed files
  - Accessing relational databases or noSQL databases
    -   MySQL, Postgres, etc. using JDBC/ODBC
    -  Hive, HBase, Cassandra, MongoDB, AWS Redshift, etc.
    
Some examples on how to create DataFrames below:

### From a sequence or a list of data

In [1]:
from pyspark.sql import SparkSession
# We create a SparkSession object (or we retrieve it if it is already created)
spark = SparkSession \
.builder \
.appName("My application") \
.config("spark.some.config.option", "some-value") \
.master("local[4]") \
.getOrCreate()
# We get the SparkContext
sc = spark.sparkContext

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/24 13:13:23 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [2]:
from pyspark.sql.functions import col,expr
# Creating a DataFrame from a range and adding two columns
df = spark.range(1,7,2).toDF("n") # from 1 increase by 2, until 7-1=6, 5+2>6 so we don't show it


df.show()
df.withColumn("n1", col("n")+1).withColumn("n2", expr("2*n")).show()
# Note that in the call to 'expr' we can include SQL code

+---+
|  n|
+---+
|  1|
|  3|
|  5|
+---+

+---+---+---+
|  n| n1| n2|
+---+---+---+
|  1|  2|  2|
|  3|  4|  6|
|  5|  6| 10|
+---+---+---+



In [3]:
# DataFrame from a list of tuples
l = [("Eric", 5.1, "Pass"),\
     ("John", 4.0, "Fail"),\
     ("Manuel", None, None)]
dfMarks = spark.createDataFrame(l, schema=["Name", "mark", "result"])
dfMarks.show()
dfMarks.printSchema()

+------+----+------+
|  Name|mark|result|
+------+----+------+
|  Eric| 5.1|  Pass|
|  John| 4.0|  Fail|
|Manuel|null|  null|
+------+----+------+

root
 |-- Name: string (nullable = true)
 |-- mark: double (nullable = true)
 |-- result: string (nullable = true)



### Creating DataFrames with a schema


When creating a DataFrame, it is a good idea to specify its schema:

  - The schema defines the names and data types of each column
  - It uses an object of type ``StructType`` to define the name and type of the columns 
  - The data types used by Spark are defined in:
      - For PySpark: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#data-types
      - For Scala: https://spark.apache.org/docs/latest/api/scala/#org.apache.spark.sql.types.package


In [4]:
from pyspark.sql.types import StructField, StructType, FloatType, StringType
from pyspark.sql import Row
# Define the DataFrame schema
schemaMarks = StructType([
    StructField("Name", StringType(), False), # can be null
    StructField("mark", FloatType(), True),
    StructField("result", StringType(), True)
    ])
    
# Create the DataFrame from a list of Row objects
rows = [Row("Eric", 5.1, "Pass"),\
         Row("John", 4.0, "Fail"),\
         Row("Manuel", None, None)]

dfMarks = spark.createDataFrame(rows, schema=schemaMarks)
dfMarks.show()
dfMarks.printSchema()

+------+----+------+
|  Name|mark|result|
+------+----+------+
|  Eric| 5.1|  Pass|
|  John| 4.0|  Fail|
|Manuel|null|  null|
+------+----+------+

root
 |-- Name: string (nullable = false)
 |-- mark: float (nullable = true)
 |-- result: string (nullable = true)



### Creating DataFrames from a text file


Each file line is stored as a row

In [5]:
# Mount first the Google Drive
#from google.colab import drive
#drive.mount('/content/gdrive')
import os
os.environ["DRIVE_DATA"] = "./data/"
dfQuijote = spark.read.text(os.environ["DRIVE_DATA"] + "quijote.txt")
dfQuijote.show(truncate=False)  # set truncate to false to extend the table with text

+---------------------------------------------------------------------------+
|value                                                                      |
+---------------------------------------------------------------------------+
|The Project Gutenberg EBook of Don Quijote, by Miguel de Cervantes Saavedra|
|                                                                           |
|This eBook is for the use of anyone anywhere at no cost and with           |
|almost no restrictions whatsoever.  You may copy it, give it away or       |
|re-use it under the terms of the Project Gutenberg License included        |
|with this eBook or online at www.gutenberg.net                             |
|                                                                           |
|                                                                           |
|Title: Don Quijote                                                         |
|                                                               

### Creating DataFrames from a CSV file (revisited)

As an example, we are going to use a file with questions and replies from Stack Exchange (https://stackexchange.com/) in Italian. 
It is a CVS file, with the following 13 fields:

  0. ``nComs`` - Number of comments of the question of the reply
  2. ``lastActivity`` - Date and hour of the last modification
  3. ``userId`` - Owner's ID 
  4. ``body`` - Text of the question or reply
  5. ``score`` - Score of the question or reply based on positive and negative votes
  6. ``creationDate`` - Creation date and hour 
  6. ``numViewed`` - Number of times viewed (null if the question has never been viewed)
  7. ``title`` - Question title (null if it is a reply)
  8. ``tags`` - Tags assigned to the question (null if there are no tags assigned)
  9. ``nAnswers`` - Number of replies related to the question (null if there are not any)
  10. ``acceptedAnswerId`` - The ID of the accepted answer (null if the question has no accepted answer)
  11. ``postType`` - Type of message: 1 question, 2 reply
  12. ``id`` - Unique message identifier

Fields are separated by the "~" symbol

#### a) Read the file and infer the schema

In [6]:
dfSEInferred = spark.read.format("csv")\
                    .option("mode", "FAILFAST")\
                    .option("sep", "~")\
                    .option("inferSchema", "true")\
                    .option("header", "false")\
                    .option("nullValue", "null")\
                    .option("compression", "bzip2")\
                    .load(os.environ["DRIVE_DATA"] +"italianPosts.csv.bz2")

Some options:

1. ``mode``: specifies what to do when it finds corrupted entries
    - ``PERMISSIVE``: sets all fields to null when a corrupted entry is found (default value)
    - ``DROPMALFORMED``: deletes the rows with corrupted entries 
    - ``FAILFAST``: returns an error when a corrupted entry is found
2. ``sep``:  field delimiter (by default ",")
3. ``inferSchema``: whether column types must be inferred (by default "false")
4. ``header``: if "true", the first line is taken as the header (by default "false")
5. ``nullValue``: character or string thar represents a NULL in the file  (by default "")
6. ``compression``: compression type (by default "none")
  
These options are similar for other types of files.

In [23]:
# Show 5 rows
dfSEInferred.show(5)

+---+--------------------+---+--------------------+---+--------------------+----+--------------------+--------------------+----+----+----+----+
|_c0|                 _c1|_c2|                 _c3|_c4|                 _c5| _c6|                 _c7|                 _c8| _c9|_c10|_c11|_c12|
+---+--------------------+---+--------------------+---+--------------------+----+--------------------+--------------------+----+----+----+----+
|  4|2013-11-11 18:21:...| 17|&lt;p&gt;The infi...| 23|2013-11-10 19:37:...|null|                null|                null|null|null|   2|1165|
|  5|2013-11-10 20:31:...| 12|&lt;p&gt;Come cre...|  1|2013-11-10 19:44:...|  61|Cosa sapreste dir...| &lt;word-choice&gt;|   1|null|   1|1166|
|  2|2013-11-10 20:31:...| 17|&lt;p&gt;Il verbo...|  5|2013-11-10 19:58:...|null|                null|                null|null|null|   2|1167|
|  1|2014-07-25 13:15:...|154|&lt;p&gt;As part ...| 11|2013-11-10 22:03:...| 187|Ironic constructi...|&lt;english-compa...|   4|1170|   

In [33]:
dfSEInferred.show(5)

+---+--------------------+---+--------------------+---+--------------------+----+--------------------+--------------------+----+----+----+----+
|_c0|                 _c1|_c2|                 _c3|_c4|                 _c5| _c6|                 _c7|                 _c8| _c9|_c10|_c11|_c12|
+---+--------------------+---+--------------------+---+--------------------+----+--------------------+--------------------+----+----+----+----+
|  4|2013-11-11 18:21:...| 17|&lt;p&gt;The infi...| 23|2013-11-10 19:37:...|null|                null|                null|null|null|   2|1165|
|  5|2013-11-10 20:31:...| 12|&lt;p&gt;Come cre...|  1|2013-11-10 19:44:...|  61|Cosa sapreste dir...| &lt;word-choice&gt;|   1|null|   1|1166|
|  2|2013-11-10 20:31:...| 17|&lt;p&gt;Il verbo...|  5|2013-11-10 19:58:...|null|                null|                null|null|null|   2|1167|
|  1|2014-07-25 13:15:...|154|&lt;p&gt;As part ...| 11|2013-11-10 22:03:...| 187|Ironic constructi...|&lt;english-compa...|   4|1170|   

In [19]:
# Find out how the schema was inferred
dfSEInferred.schema

StructType([StructField('_c0', IntegerType(), True), StructField('_c1', TimestampType(), True), StructField('_c2', IntegerType(), True), StructField('_c3', StringType(), True), StructField('_c4', IntegerType(), True), StructField('_c5', TimestampType(), True), StructField('_c6', IntegerType(), True), StructField('_c7', StringType(), True), StructField('_c8', StringType(), True), StructField('_c9', IntegerType(), True), StructField('_c10', IntegerType(), True), StructField('_c11', IntegerType(), True), StructField('_c12', IntegerType(), True)])

In [34]:
# Another way of getting the same result
dfSEInferred.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- _c1: timestamp (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: string (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: timestamp (nullable = true)
 |-- _c6: integer (nullable = true)
 |-- _c7: string (nullable = true)
 |-- _c8: string (nullable = true)
 |-- _c9: integer (nullable = true)
 |-- _c10: integer (nullable = true)
 |-- _c11: integer (nullable = true)
 |-- _c12: integer (nullable = true)



#### b) Read the file and specify the schema

In [7]:
from pyspark.sql.types import *
# We first create a list with each column header
# Note: avoid spaces and non-ascii characters on column names
header = (["nComs", "lastActivity", "userId", 
            "body", "score", "creationDate", "numViewed", "title", 
            "tags", "nAnswers", "acceptedAnswerId", "postType", "id"])
            
# Define the schema for the elements of the table
# StructType -> Defines a schema for the DF from a list of StructFields
# StructField -> Defines the name and type of each column, and whether it is nullable or not (True field)
dfSE_Schema = StructType([
  StructField(header[0], IntegerType(), True),
  StructField(header[1], TimestampType(), True),
  StructField(header[2], LongType(), True),
  StructField(header[3], StringType(), True),
  StructField(header[4], IntegerType(), True),
  StructField(header[5], TimestampType(), True),
  StructField(header[6], IntegerType(), True),
  StructField(header[7], StringType(), True),
  StructField(header[8], StringType(), True),
  StructField(header[9], IntegerType(), True),
  StructField(header[10], LongType(), True),
  StructField(header[11], ByteType(), True),
  StructField(header[12], LongType(), True)
  ])
  
dfSE = spark.read.format("csv")\
                    .option("mode", "FAILFAST")\
                    .option("sep", "~")\
                    .option("inferSchema", "false")\
                    .option("header", "false")\
                    .option("nullValue", "null")\
                    .option("compression", "bzip2")\
                    .schema(dfSE_Schema)\
                    .load(os.environ["DRIVE_DATA"] +"italianPosts.csv.bz2")
dfSE.cache()                    

DataFrame[nComs: int, lastActivity: timestamp, userId: bigint, body: string, score: int, creationDate: timestamp, numViewed: int, title: string, tags: string, nAnswers: int, acceptedAnswerId: bigint, postType: tinyint, id: bigint]

In [37]:
dfSE.sort("id").show()

+-----+--------------------+------+--------------------+-----+--------------------+---------+--------------------+--------------------+--------+----------------+--------+---+
|nComs|        lastActivity|userId|                body|score|        creationDate|numViewed|               title|                tags|nAnswers|acceptedAnswerId|postType| id|
+-----+--------------------+------+--------------------+-----+--------------------+---------+--------------------+--------------------+--------+----------------+--------+---+
|    2|2013-11-09 09:20:...|     8|&lt;p&gt;What is ...|   20|2013-11-05 20:22:...|      204|What are the rule...|&lt;word-choice&g...|       4|               2|       1|  1|
|    9|2013-11-05 20:44:...|    17|&lt;p&gt;Translat...|   18|2013-11-05 20:36:...|     null|                null|                null|    null|            null|       2|  2|
|    0|2013-11-05 20:37:...|     6|&lt;p&gt;[don't k...|    6|2013-11-05 20:37:...|     null|                null|           

In [38]:
dfSE.printSchema()

root
 |-- nComs: integer (nullable = true)
 |-- lastActivity: timestamp (nullable = true)
 |-- userId: long (nullable = true)
 |-- body: string (nullable = true)
 |-- score: integer (nullable = true)
 |-- creationDate: timestamp (nullable = true)
 |-- numViewed: integer (nullable = true)
 |-- title: string (nullable = true)
 |-- tags: string (nullable = true)
 |-- nAnswers: integer (nullable = true)
 |-- acceptedAnswerId: long (nullable = true)
 |-- postType: byte (nullable = true)
 |-- id: long (nullable = true)



## Basic operations with DataFrames

### Show rows

In [39]:
# show(n) shows the first n rows (by default, n=20)
dfSE.show(5)

+-----+--------------------+------+--------------------+-----+--------------------+---------+--------------------+--------------------+--------+----------------+--------+----+
|nComs|        lastActivity|userId|                body|score|        creationDate|numViewed|               title|                tags|nAnswers|acceptedAnswerId|postType|  id|
+-----+--------------------+------+--------------------+-----+--------------------+---------+--------------------+--------------------+--------+----------------+--------+----+
|    4|2013-11-11 18:21:...|    17|&lt;p&gt;The infi...|   23|2013-11-10 19:37:...|     null|                null|                null|    null|            null|       2|1165|
|    5|2013-11-10 20:31:...|    12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|       61|Cosa sapreste dir...| &lt;word-choice&gt;|       1|            null|       1|1166|
|    2|2013-11-10 20:31:...|    17|&lt;p&gt;Il verbo...|    5|2013-11-10 19:58:...|     null|                null|      

In [41]:
# Say that we do not want to truncate the long fields
dfSE.show(5, truncate=False)

+-----+-----------------------+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [42]:
# take(n) returns the first n rows as a Python list of Row objects
list = dfSE.take(5)
print(list[1])
print("\n")
# collect() returns the DataFrame as a Python list of Row objects
# Warning: if the DataFrame is too large, it might collapse the Driver!
list2 = dfSE.collect()
print(list2[10])

Row(nComs=5, lastActivity=datetime.datetime(2013, 11, 10, 20, 31, 0, 177000), userId=12, body="&lt;p&gt;Come credo sia conosciuto da tutti quelli che usano viaggiare con l'automobile, molti italiani hanno uno strano rapporto con gli abbaglianti; alcuni li amano così tanto che preferiscono mantenerli sempre accesi, altri invece li usano per segnalare, se non addirittura per comunicare informazioni di vario genere, dalla presenza di autovelox alla protesta per presunte violazioni del codice della strada.&lt;/p&gt;&lt;p&gt;Al di lá delle considerazioni e dei commenti circa queste abitudini, mi piacerebbe sapere se il verbo &quot;sfanagliare&quot; è normalmente usato, e compreso, in tutte le regioni italiane o se, magari, ci sono altri verbi in uso, purchè simpatici come quello.&lt;/p&gt;&lt;p&gt;Laddove qualcuno non avesse compreso l'uso del aforementioned verbo, ecco un esempio:&lt;/p&gt;&lt;blockquote&gt;  &lt;p&gt;&quot;Ehi!&quot; - dice il marito a sua moglie - &quot;Quello li mi sta 

In [8]:
import os
# sample(withReplacement, fraction, seed=None) returns a new Dataframe with a fraction of the original rows
dfSESampled = dfSE.sample(False, 0.1, seed=False)  # take 10% of data, withReplacement set to false, to avoid repetition because we take values randomly from the data set and we may repeat the data, True if we want to show repeated data.
dfSESampled.show(5)
print("Original Number of rows = {0}; Number of sampled rows = {1}".format(dfSE.count(), dfSESampled.count()))

+-----+--------------------+------+--------------------+-----+--------------------+---------+--------------------+--------------------+--------+----------------+--------+----+
|nComs|        lastActivity|userId|                body|score|        creationDate|numViewed|               title|                tags|nAnswers|acceptedAnswerId|postType|  id|
+-----+--------------------+------+--------------------+-----+--------------------+---------+--------------------+--------------------+--------+----------------+--------+----+
|    2|2013-11-10 20:31:...|    17|&lt;p&gt;Il verbo...|    5|2013-11-10 19:58:...|     null|                null|                null|    null|            null|       2|1167|
|    0|2013-11-11 10:58:...|    18|&lt;p&gt;Wow, wha...|    5|2013-11-11 10:58:...|     null|                null|                null|    null|            null|       2|1174|
|    0|2013-11-11 18:20:...|   132|&lt;p&gt;I would ...|    8|2013-11-11 14:04:...|     null|                null|      

In [9]:
# limit(n) limits the number of rows calculated to n
dfSE_10rows = dfSE.sample(False, 0.1, seed=None).limit(10)
print("Number of sampled rows = {0}".format(dfSE_10rows.count()))
dfSE_10rows.show()

Number of sampled rows = 10
+-----+--------------------+------+--------------------+-----+--------------------+---------+--------------------+--------------------+--------+----------------+--------+----+
|nComs|        lastActivity|userId|                body|score|        creationDate|numViewed|               title|                tags|nAnswers|acceptedAnswerId|postType|  id|
+-----+--------------------+------+--------------------+-----+--------------------+---------+--------------------+--------------------+--------+----------------+--------+----+
|    4|2013-11-11 18:21:...|    17|&lt;p&gt;The infi...|   23|2013-11-10 19:37:...|     null|                null|                null|    null|            null|       2|1165|
|    0|2013-11-10 22:15:...|    70|&lt;p&gt;&lt;em&g...|    3|2013-11-10 22:15:...|     null|                null|                null|    null|            null|       2|1169|
|    0|2013-11-12 11:24:...|    63|&lt;p&gt;Comparin...|    3|2013-11-11 12:58:...|       60

### Execute an operation on each row
The method `foreach` applies a function to each row

- The DataFrame is not modified and no other DataFrames are created
- `foreach` is executed in the Workers

In [10]:
def printid(f):
    print(f["id"])
    
# In theory, this code should print all values of the 'id' column.
# Due to the way the notebook manages tasks, it is not possible to see any output.
# Run it on a pyspark-shell to see the output.
dfSE_10rows.foreach(printid)


1165
1169
1179
1204
1219
1233
1255
1281
1293
1299


### Select columns

In [11]:
# Creates a new DataFrame by selecting columns by name
dfIdBody = dfSE.select("id", "body")
dfIdBody.show(5)

print("The idBody object is of type {0}".format(type(dfIdBody)))

+----+--------------------+
|  id|                body|
+----+--------------------+
|1165|&lt;p&gt;The infi...|
|1166|&lt;p&gt;Come cre...|
|1167|&lt;p&gt;Il verbo...|
|1168|&lt;p&gt;As part ...|
|1169|&lt;p&gt;&lt;em&g...|
+----+--------------------+
only showing top 5 rows

The idBody object is of type <class 'pyspark.sql.dataframe.DataFrame'>


In [12]:
# Another way of specifying the columns to select
dfIdBody2 = dfSE.select(dfSE.id, dfSE.body)
dfIdBody2.show(5)

+----+--------------------+
|  id|                body|
+----+--------------------+
|1165|&lt;p&gt;The infi...|
|1166|&lt;p&gt;Come cre...|
|1167|&lt;p&gt;Il verbo...|
|1168|&lt;p&gt;As part ...|
|1169|&lt;p&gt;&lt;em&g...|
+----+--------------------+
only showing top 5 rows



In [13]:
# It is also possible to specify objects of Column type...
from pyspark.sql.functions import col

colId = col("id")
colCreateDate = col("creationDate")
print("The colId object is of type {0}".format(type(colId)))
print("The colCreateDate object is of type {0}".format(type(colCreateDate)))

The colId object is of type <class 'pyspark.sql.column.Column'>
The colCreateDate object is of type <class 'pyspark.sql.column.Column'>


In [33]:
# ... and create a DataFrame from Column objects, by renaming the columns
dfIdBodyDate = dfSE.select(colId, 
                              colCreateDate.alias("Creation_date"), 
                              dfSE.body.alias("Content"))
dfIdBodyDate.show(5)

+----+--------------------+--------------------+
|  id|       Creation_date|             Content|
+----+--------------------+--------------------+
|1165|2013-11-10 19:37:...|&lt;p&gt;The infi...|
|1166|2013-11-10 19:44:...|&lt;p&gt;Come cre...|
|1167|2013-11-10 19:58:...|&lt;p&gt;Il verbo...|
|1168|2013-11-10 22:03:...|&lt;p&gt;As part ...|
|1169|2013-11-10 22:15:...|&lt;p&gt;&lt;em&g...|
+----+--------------------+--------------------+
only showing top 5 rows



#### Select columns by using expressions

To select columns using SQL expressions

In [34]:
# We can use more complex expressions
dfSE.selectExpr("*", # Select all columns and set ValidReply to True for those with, at least, one reply.
                "(nAnswers IS NOT NULL) as ValidReply").show()

+-----+--------------------+------+--------------------+-----+--------------------+---------+--------------------+--------------------+--------+----------------+--------+----+----------+
|nComs|        lastActivity|userId|                body|score|        creationDate|numViewed|               title|                tags|nAnswers|acceptedAnswerId|postType|  id|ValidReply|
+-----+--------------------+------+--------------------+-----+--------------------+---------+--------------------+--------------------+--------+----------------+--------+----+----------+
|    4|2013-11-11 18:21:...|    17|&lt;p&gt;The infi...|   23|2013-11-10 19:37:...|     null|                null|                null|    null|            null|       2|1165|     false|
|    5|2013-11-10 20:31:...|    12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|       61|Cosa sapreste dir...| &lt;word-choice&gt;|       1|            null|       1|1166|      true|
|    2|2013-11-10 20:31:...|    17|&lt;p&gt;Il verbo...|    5|201

In [14]:
from pyspark.sql.functions import expr
# Same DataFrame as before but using expressions
dfIdDateBodyExpr = dfSE.select(
                           expr("id AS ID"), 
                           expr('creationDate AS Creation_date'), 
                           expr("body AS Content"))
dfIdDateBodyExpr.show(5)


+----+--------------------+--------------------+
|  ID|       Creation_date|             Content|
+----+--------------------+--------------------+
|1165|2013-11-10 19:37:...|&lt;p&gt;The infi...|
|1166|2013-11-10 19:44:...|&lt;p&gt;Come cre...|
|1167|2013-11-10 19:58:...|&lt;p&gt;Il verbo...|
|1168|2013-11-10 22:03:...|&lt;p&gt;As part ...|
|1169|2013-11-10 22:15:...|&lt;p&gt;&lt;em&g...|
+----+--------------------+--------------------+
only showing top 5 rows



### Rename, add and delete columns


In [15]:
# Rename the creationDate column
dfSE = dfSE.withColumnRenamed("creationDate", "Creation_date")
dfSE.cache()
dfSE.select("Creation_date", 
            dfSE.numViewed.alias("Number_of_visits"), 
            "score", 
            "postType").show(truncate=False)

+-----------------------+----------------+-----+--------+
|Creation_date          |Number_of_visits|score|postType|
+-----------------------+----------------+-----+--------+
|2013-11-10 19:37:54.187|null            |23   |2       |
|2013-11-10 19:44:53.797|61              |1    |1       |
|2013-11-10 19:58:02.1  |null            |5    |2       |
|2013-11-10 22:03:41.027|187             |11   |1       |
|2013-11-10 22:15:17.693|null            |3    |2       |
|2013-11-10 22:17:22.38 |null            |8    |2       |
|2013-11-11 09:51:11.22 |null            |3    |2       |
|2013-11-11 10:09:05.117|null            |1    |2       |
|2013-11-11 10:28:12.613|122             |5    |1       |
|2013-11-11 10:58:02.62 |null            |5    |2       |
|2013-11-11 11:31:02.343|114             |4    |1       |
|2013-11-11 11:39:12.703|58              |3    |1       |
|2013-11-11 11:57:03.723|null            |6    |2       |
|2013-11-11 12:00:25.583|null            |1    |2       |
|2013-11-11 12

In [16]:
# Add a new column 'ones' with all its values set to 1
from pyspark.sql.functions import lit
# lit transforms a literal in Python to Spark internal format
# (in this example, IntegerType)
dfSE = dfSE.withColumn("ones", lit(1))  # we can specify values we want to insert with lit, we can do lit(dfSE.score * 2) ....
dfSE.show(5)

+-----+--------------------+------+--------------------+-----+--------------------+---------+--------------------+--------------------+--------+----------------+--------+----+----+
|nComs|        lastActivity|userId|                body|score|       Creation_date|numViewed|               title|                tags|nAnswers|acceptedAnswerId|postType|  id|ones|
+-----+--------------------+------+--------------------+-----+--------------------+---------+--------------------+--------------------+--------+----------------+--------+----+----+
|    4|2013-11-11 18:21:...|    17|&lt;p&gt;The infi...|   23|2013-11-10 19:37:...|     null|                null|                null|    null|            null|       2|1165|   1|
|    5|2013-11-10 20:31:...|    12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|       61|Cosa sapreste dir...| &lt;word-choice&gt;|       1|            null|       1|1166|   1|
|    2|2013-11-10 20:31:...|    17|&lt;p&gt;Il verbo...|    5|2013-11-10 19:58:...|     null|  

In [18]:
# Removes a column using drop
dfSE = dfSE.drop(col("ones"))
dfSE.columns

['nComs',
 'lastActivity',
 'userId',
 'body',
 'score',
 'Creation_date',
 'numViewed',
 'title',
 'tags',
 'nAnswers',
 'acceptedAnswerId',
 'postType',
 'id']

### Delete null and duplicated values

In [19]:
# Remove all rows that have null on any of their columns
dfNoNulls = dfSE.dropna("any")
print("Initial number or rows: {0}; number of non null rows: {1}"
       .format(dfSE.count(), dfNoNulls.count()))


Initial number or rows: 1261; number of non null rows: 222


In [20]:
# Remove rows that have null on all their columns
dfNeitherNull = dfSE.dropna("all")
print("Number of rows with all columns set to null: {0}"
       .format(dfSE.count() - dfNeitherNull.count()))

Number of rows with all columns set to null: 0


In [41]:
# Remove duplicated rows
dfWithoutDuplicates = dfSE.dropDuplicates()
print("Number of duplicated rows: {0}"
       .format(dfSE.count() - dfWithoutDuplicates.count()))

Number of duplicated rows: 0


In [21]:
# Remove rows when a given column is duplicated
dfWithoutDuplicatedUser = dfSE.dropDuplicates(["userId"])
print("Number of unique users: {0}"
       .format(dfWithoutDuplicatedUser.count()))

Number of unique users: 218


In [22]:
# Other examples
dfNoNullnumViewedAcceptedAnswerId = dfSE.dropna("any", subset=["numViewed", "acceptedAnswerId"]) # null in any of the columns, both shouldn't be null
print("Number of rows with numViewed AND acceptedAnswerId not null: {0}"
       .format(dfNoNullnumViewedAcceptedAnswerId.count()))

dfNoNullnumViewedAcceptedAnswerId = dfSE.dropna("all", subset=["numViewed", "acceptedAnswerId"]) # null in all columns, 1 of the columns shouldn't be null
print("Number of rows with numViewed OR acceptedAnswerId not null: {0}"
       .format(dfNoNullnumViewedAcceptedAnswerId.count()))

Number of rows with numViewed AND acceptedAnswerId not null: 222
Number of rows with numViewed OR acceptedAnswerId not null: 374


### Replacing values

In [23]:
# Replace with '0' all null values in the numVistas and nAnswers fields
dfSE = dfSE.fillna(0, subset=["numViewed", "nAnswers"])
dfSE.show(5)

+-----+--------------------+------+--------------------+-----+--------------------+---------+--------------------+--------------------+--------+----------------+--------+----+
|nComs|        lastActivity|userId|                body|score|       Creation_date|numViewed|               title|                tags|nAnswers|acceptedAnswerId|postType|  id|
+-----+--------------------+------+--------------------+-----+--------------------+---------+--------------------+--------------------+--------+----------------+--------+----+
|    4|2013-11-11 18:21:...|    17|&lt;p&gt;The infi...|   23|2013-11-10 19:37:...|        0|                null|                null|       0|            null|       2|1165|
|    5|2013-11-10 20:31:...|    12|&lt;p&gt;Come cre...|    1|2013-11-10 19:44:...|       61|Cosa sapreste dir...| &lt;word-choice&gt;|       1|            null|       1|1166|
|    2|2013-11-10 20:31:...|    17|&lt;p&gt;Il verbo...|    5|2013-11-10 19:58:...|        0|                null|      

In [24]:
# Replace the value 1170 with 3000 in columns "id" and "acceptedAnswerId"
dfSE.select("id", "acceptedAnswerId").show(10)
dfSE.replace(1170, 3000, subset=["id", "acceptedAnswerId"])\
    .select("id", "acceptedAnswerId")\
    .show(10)

+----+----------------+
|  id|acceptedAnswerId|
+----+----------------+
|1165|            null|
|1166|            null|
|1167|            null|
|1168|            1170|
|1169|            null|
|1170|            null|
|1171|            null|
|1172|            null|
|1173|            1181|
|1174|            null|
+----+----------------+
only showing top 10 rows

+----+----------------+
|  id|acceptedAnswerId|
+----+----------------+
|1165|            null|
|1166|            null|
|1167|            null|
|1168|            3000|
|1169|            null|
|3000|            null|
|1171|            null|
|1172|            null|
|1173|            1181|
|1174|            null|
+----+----------------+
only showing top 10 rows



## Saving DataFrames

As for reading, Spark can save DateFrames in multiple formats:

- CSV, JSON, Parquet, Hadoop...

It can write them as well on a database

In [26]:
# Save the dfSE DataFrame in JSON format
#dfSE.write.format("json").mode("overwrite").save("/content/dfSE.json")
dfSE.write.json(os.environ["DRIVE_DATA"] + "dfSE.json",mode="overwrite")  # save folder in repository with the possibility of over writing it

#!mv /content/dfSE.json "$DRIVE_DATA" 

In [27]:
!ls -alh "$DRIVE_DATA"/dfSE.json

total 1.4M
drwxr-xr-x 1 root root 4.0K Nov 24 13:31 .
drwxrwxrwx 1 root root 4.0K Nov 24 13:31 ..
-rw-r--r-- 1 root root    8 Nov 24 13:31 ._SUCCESS.crc
-rw-r--r-- 1 root root  11K Nov 24 13:31 .part-00000-a415ab7e-1e5f-435c-bfbd-779e64554f13-c000.json.crc
-rw-r--r-- 1 root root    0 Nov 24 13:31 _SUCCESS
-rw-r--r-- 1 root root 1.4M Nov 24 13:31 part-00000-a415ab7e-1e5f-435c-bfbd-779e64554f13-c000.json


In [29]:
# Save the DataFrame using Parquet
dfSE.write.format("parquet")\
    .mode("overwrite")\
    .save(os.environ["DRIVE_DATA"] + "dfSE.parquet")

In [30]:
# Parquet uses by default the Snappy compressed format
!ls -alh "$DRIVE_DATA"/dfSE.parquet

total 624K
drwxr-xr-x 1 root root 4.0K Nov 24 13:32 .
drwxrwxrwx 1 root root 4.0K Nov 24 13:32 ..
-rw-r--r-- 1 root root    8 Nov 24 13:32 ._SUCCESS.crc
-rw-r--r-- 1 root root 4.9K Nov 24 13:32 .part-00000-5a57b70c-9418-4dc6-99b3-5e03f739abac-c000.snappy.parquet.crc
-rw-r--r-- 1 root root    0 Nov 24 13:32 _SUCCESS
-rw-r--r-- 1 root root 615K Nov 24 13:32 part-00000-5a57b70c-9418-4dc6-99b3-5e03f739abac-c000.snappy.parquet


It will create as many files as there are partitions in the DataFrame

In [31]:
dfSE2 = dfSE.repartition(2)
# Save the DataFrame using Parquet, with gzip compression
# Parquet is a columnar format that is supported by many other data processing systems. Spark SQL provides support for both reading and writing Parquet files that automatically preserves the schema of the original data. When reading Parquet files, all columns are automatically converted to be nullable for compatibility reasons.
dfSE2.write.format("parquet")\
     .mode("overwrite")\
     .option("compression", "gzip")\
     .save(os.environ["DRIVE_DATA"] + "/dfSE2.parquet")

In [32]:
!ls -alh "$DRIVE_DATA"/dfSE2.parquet

total 424K
drwxr-xr-x 1 root root 4.0K Nov 24 13:37 .
drwxrwxrwx 1 root root 4.0K Nov 24 13:37 ..
-rw-r--r-- 1 root root    8 Nov 24 13:37 ._SUCCESS.crc
-rw-r--r-- 1 root root 1.6K Nov 24 13:37 .part-00000-ff5ec56e-a293-454e-bf2e-6d391ba44d0c-c000.gz.parquet.crc
-rw-r--r-- 1 root root 1.7K Nov 24 13:37 .part-00001-ff5ec56e-a293-454e-bf2e-6d391ba44d0c-c000.gz.parquet.crc
-rw-r--r-- 1 root root    0 Nov 24 13:37 _SUCCESS
-rw-r--r-- 1 root root 203K Nov 24 13:37 part-00000-ff5ec56e-a293-454e-bf2e-6d391ba44d0c-c000.gz.parquet
-rw-r--r-- 1 root root 212K Nov 24 13:37 part-00001-ff5ec56e-a293-454e-bf2e-6d391ba44d0c-c000.gz.parquet


### Partitioning

Spark can partition and save a file using the value of a given column

- A directory is created for each different value in the partitioning column
    - All data associated to that value are stored in that directory
- It simplifies the access to the values associated to a given key


In [36]:
# Save our DataFrame partitioned by the userID field (using Parquet)
dfSE.write.format("parquet")\
    .mode("overwrite")\
    .partitionBy("userId")\
    .save(os.environ["DRIVE_DATA"] + "dfSE-partition.parquet")


In [34]:
#!ls -lh "$DRIVE_DATA"dfSE-partition.parquet
!ls -lh "$DRIVE_DATA"dfSE-partition.parquet/userId=10
#rm -rf "$DRIVE_DATA"dfSE-partition.parquet

total 8.0K
-rw-r--r-- 1 root root 4.4K Nov 24 13:42 part-00000-e0845598-7893-4b65-9bd5-a4d7d6526e58.c000.snappy.parquet


In [35]:
#!ls -lh "$DRIVE_DATA"dfSE-partition.parquet
!ls -lh "$DRIVE_DATA"dfSE-partition.parquet/userId=10
#rm -rf "$DRIVE_DATA"dfSE-partition.parquet

total 8.0K
-rw-r--r-- 1 root root 4.4K Nov 24 13:42 part-00000-e0845598-7893-4b65-9bd5-a4d7d6526e58.c000.snappy.parquet




---

# Exercises


## Exercise 3.1: Word count

Count the number of words *per line* in the $DRIVE_DATA/quijote.txt file. 

Repeat the exercise but this time counting the number of words *in the whole file*.

# Exercise 3.1 Solution: 

In [2]:
from pyspark.sql import SparkSession
import os

os.environ["DRIVE_DATA"] = "./data/"
spark = SparkSession.builder.appName("ex3").getOrCreate()

# sc = spark.sparkContext

/usr/local/lib/python3.9/dist-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/12/20 12:29:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.sql import functions as F
# so that we can use the F.split() function.


file_read = spark.read.text(os.environ["DRIVE_DATA"] + "quijote.txt")
file_read.show(5, truncate=False)

+---------------------------------------------------------------------------+
|value                                                                      |
+---------------------------------------------------------------------------+
|The Project Gutenberg EBook of Don Quijote, by Miguel de Cervantes Saavedra|
|                                                                           |
|This eBook is for the use of anyone anywhere at no cost and with           |
|almost no restrictions whatsoever.  You may copy it, give it away or       |
|re-use it under the terms of the Project Gutenberg License included        |
+---------------------------------------------------------------------------+
only showing top 5 rows



In [9]:
def split_count(line):
    # if line is []:
    #print("line : ", line[0])
    #print("line : ", line.to)
    return F.size(F.split(line, " "))
    #return F.split(line, " ")

#num_words_per_line = file_read.select(F.regexp_extract(col("value"), "[a-z']*", 0)).withColumn("num_words", lit(split_count(F.trim(file_read.value)))).select(F.col("value").alias("lines"), F.col("num_words"))
#num_words_per_line.printSchema()
#num_words_per_line.show(5, truncate=False)
#file_read.select(F.regexp_extract(F.col("value"), "[a-z]*", 0)).show()
# file_read.select(F.regexp_extract(F.col("value"), "([a-zA-Z]+)", 1)).show()
#file_read.withColumn('lines', F.expr(r"regexp_extract_all(value, '[a-z]*(\\d+)', 1)")).show() # show numbers
#file_read.withColumn('lines', F.expr(r"regexp_extract_all(value, '([a-zA-Z]+)', 1)")).show(truncate=False)
num_words_per_line = file_read.select(F.expr(r"regexp_extract_all(value, '([a-zA-ZÀ-ÿ]+[-_.:/@]*[a-zA-ZÀ-ÿ]*)')").alias("lines")).withColumn("num_words", F.lit(F.size(F.col("lines"))))
num_words_per_line.withColumn("index", F.monotonically_increasing_id()+1).filter(F.col("index").between(390, 400)).show(50,truncate=False)

+--------------------------------+---------+-----+
|lines                           |num_words|index|
+--------------------------------+---------+-----+
|[]                              |0        |390  |
|[no, dirá, mofante, algu-:]     |4        |391  |
|[Qué, don, Álvaro, de, Lu-]     |5        |392  |
|[qué, Anibal, el, de, Carta-]   |5        |393  |
|[qué, rey, Francisco, en, Espa-]|5        |394  |
|[se, queja, de, la, Fortu-]     |5        |395  |
|[Pues, al, cielo, no, le, plu-] |6        |396  |
|[que, salieses, tan, ladi-]     |4        |397  |
|[como, el, negro, Juan, Lati-]  |5        |398  |
|[hablar, latines, rehú-.]       |3        |399  |
|[No, me, despuntes, de, agu-]   |5        |400  |
+--------------------------------+---------+-----+



# Exercise 3.2 Solution: 

In [10]:
num_words_per_line.agg({'num_words': 'sum'}).show()

+--------------+
|sum(num_words)|
+--------------+
|        384233|
+--------------+



In [297]:
### num_words_per_line.filter(num_words_per_line.num_words>1).select(F.sum('num_words')).collect() 
num_words_per_line.select(F.sum('num_words')).collect() 
#num_words_per_line.filter(num_words_per_line.num_words>1).show(truncate=False)

[Row(sum(num_words)=384219)]